# housekeeping

In [1]:
import re
from selenium import webdriver # webdriver 操作一般用
from selenium.webdriver.chrome import service as fs # Chrome を driver として設定する用
from selenium.webdriver.chrome.options import Options # headless モードで作業する用
from selenium.webdriver.common.by import By # webpage の参照したい位置を特定する用
from bs4 import BeautifulSoup

In [2]:
# chromedriver の path を指定する
DRIVER_PATH = '/Users/domolm/.pyenv/versions/3.10.8/lib/python3.10/site-packages/selenium/chromedriver'

# Chrome を driver として設定するための準備
chrome_service = fs.Service(executable_path=DRIVER_PATH)

# Selenium に慣れる

In [3]:
# 通常ブラウザで Google を開く
# driver = webdriver.Chrome(service=chrome_service)
# driver.get('https://google.com')

# ブラウザを消去
# driver.quit()

In [4]:
# headless モードで起動する
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

# 起動するときは必ず driver の定義から行う必要がある
driver = webdriver.Chrome(options=options, service=chrome_service)

# テストで任天堂の情報を取ってこよう
url = 'https://www.nintendo.com/'
driver.get(url)
print(driver.page_source)

# driver を shut down
driver.quit()


<html lang="en-us" class="js-focus-visible svg cssanimations flexboxlegacy fontface csstransforms supports csstransforms3d csstransitions inlinesvg alps-os-mac" data-js-focus-visible="" style="--app-height:1200px;"><head><meta charset="utf-8"><title>Nintendo Official Site: Consoles, Games, News, and More</title><link rel="canonical" href="https://www.nintendo.com/"><meta http-equiv="Accept-CH" content="DPR, Width"><meta name="description" content="Visit the official Nintendo site to shop for Nintendo Switch™ systems and video games, read the latest news, find fun gear and gifts with a Nintendo twist, and much more."><meta name="twitter:card" content="summary_large_image"><meta name="twitter:site" content="@nintendoamerica"><meta property="og:type" content="website"><meta property="og:title" name="twitter:title" content="Nintendo Official Site: Consoles, Games, News, and More"><meta property="og:description" name="twitter:description" content="Visit the official Nintendo site to shop fo

In [5]:
driver = webdriver.Chrome(options=options, service=chrome_service)
driver.get(url)

# to get the page's title
print(driver.title)

driver.quit()


Nintendo Official Site: Consoles, Games, News, and More


In [6]:
driver = webdriver.Chrome(options=options, service=chrome_service)
driver.get(url)

# to get the current URL 
# (this can be useful when there are redirections on the website and you need the final URL)
print(driver.current_url)

driver.quit()

https://www.nintendo.com/


# JavaScript を読み込んだ状態でデータ取得

In [7]:
driver = webdriver.Chrome(options=options, service=chrome_service)
url = 'https://www3.nhk.or.jp/news/easy/k10013897051000/k10013897051000.html'
driver.get(url)

# 文章の生データを持ってくる
# article = driver.find_element(By.XPATH, '//*[@id="js-article-body"]')

# HTML を文字コードを UTF-8 に変換してから取得する。
html = driver.page_source.encode('utf-8')

driver.quit()


# 取得したデータをパースする

In [8]:
# BeautifulSoup で扱えるようにする
soup = BeautifulSoup(html, 'html.parser')

# 邪魔な rt を消す
忌まわしい... これにどれだけ苦労させられたか... まじこのサイト [(ニュースのスクレイピングでタイピング練習)](https://pasero.net/~mako/blog/s/2364) 神...

In [9]:
for s in soup(['rt']):
    # BeautifulSoup()の関数:decompose() = 文字削除
    s.decompose()

# text を抽出

In [10]:
# id で特定の要素を取り出す
article_raw = soup.select_one('#js-article-body').text
article_raw

'\n養殖で人が育てる魚の餌には、魚を粉にした魚粉を使っています。養殖を行う国が増えて、魚粉の値段が高くなっています。\nこのため、お茶の水女子大学などのグループは、アメリカミズアブという虫を餌に使う研究をしています。この虫は大きくなるのが早くて、卵から3週間ぐらいあとで餌に使うことができます。\n東京で19日、研究について発表がありました。グループの人は、餌の半分をアメリカミズアブに変えても、魚は同じように大きくなったと話しました。\n会場では、この虫を使った餌で育てた魚と、今までの餌で育てた魚の刺身を食べてみました。食べた人は「味は同じだと思います。おいしかったです」と話しました。\nグループの人は「虫をいろいろなことに使って、食べ物が足りなくなる心配がないようにしていきたいです」と話しました。\n\n\n'

# 余計な文字を削除する

In [12]:
article = re.sub('\n', '', article_raw)
article = re.split('(?<=。)', article)
article = [s for s in article if s != '']
article

['養殖で人が育てる魚の餌には、魚を粉にした魚粉を使っています。',
 '養殖を行う国が増えて、魚粉の値段が高くなっています。',
 'このため、お茶の水女子大学などのグループは、アメリカミズアブという虫を餌に使う研究をしています。',
 'この虫は大きくなるのが早くて、卵から3週間ぐらいあとで餌に使うことができます。',
 '東京で19日、研究について発表がありました。',
 'グループの人は、餌の半分をアメリカミズアブに変えても、魚は同じように大きくなったと話しました。',
 '会場では、この虫を使った餌で育てた魚と、今までの餌で育てた魚の刺身を食べてみました。',
 '食べた人は「味は同じだと思います。',
 'おいしかったです」と話しました。',
 'グループの人は「虫をいろいろなことに使って、食べ物が足りなくなる心配がないようにしていきたいです」と話しました。']